# Instalo e importo librerías

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u382-ga-1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from nltk import word_tokenize
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Google Drive

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Creo sesión de spark y leo csv

In [ ]:
downloaded = drive.CreateFile({'id':"10xgOf2rORcGlcKPME3cHGHssXGzNemer"})
downloaded.GetContentFile('GooglePlayStore_User_Reviews.csv')

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.option("delimiter", ",").option("escape", '"').csv('GooglePlayStore_User_Reviews.csv', header=True, inferSchema=True)
user_reviews = df.rdd

# Ejercicio 11

Realizar un análisis de stopwords de las reviews. Dada la frecuencia de los tokens de las reviews, mostrar los 30 tokens más frecuentes y listar del total de tokens cuales son stopwords utilizando nltk. (⭐⭐)

Filtro las reviews nan ya que las considero un error del archivo (y además no me sirven para analizar las palabras de cada una).

Uso distinct para eliminar reviews que se encuentran repetidas (provoca que se cuenten más de una vez palabras o tokens).

Mapeo para quedarme solo con las reviews.

Uso flatMap para quedarme con todos los tokens de cada review (habrá repeticiones por usar flatMap). Como tokenizer utilizo el de nltk.

Mapeo para quedarme los tokens (todos en minúscula) como clave y 1 como valor, para desppués sumarlos con reduceByKey, quedándome así con todos los tokens y sus frecuencias.

Cacheo porque haré más de una acción sobre esto (tomo los 30 más frecuentes y luego filtro por stopwords).



In [ ]:
tokens_by_freq = user_reviews.filter(lambda x: (x.Translated_Review != 'nan') & (x.Translated_Review is not None)).distinct()\
.map(lambda x: x.Translated_Review).flatMap(lambda x: word_tokenize(x))\
.map(lambda x: (x.lower(), 1)).reduceByKey(lambda x,y: x+y).cache()

In [ ]:
tokens_by_freq.takeOrdered(30, lambda x: -x[1])  #puedo hacer map sortBy(x[1]) o dar vuelta clave-valor y luego sortByKey, para después take(30)

[('.', 48630),
 ('i', 28375),
 (',', 20734),
 ('!', 10804),
 ('it', 9065),
 ('game', 6403),
 ('the', 5156),
 ("'s", 4118),
 ('good', 4105),
 ('like', 4100),
 ('app', 3897),
 ('this', 3854),
 ('great', 3779),
 ('love', 3535),
 ('get', 3463),
 ('time', 3337),
 ('...', 2995),
 ("n't", 2958),
 ('?', 2751),
 ('would', 2523),
 ('really', 2335),
 ('even', 2151),
 ('not', 2056),
 ('ca', 2019),
 ('update', 1957),
 ('phone', 1935),
 ('you', 1853),
 ('work', 1824),
 ("'m", 1823),
 ('please', 1768)]

In [ ]:
tokens_by_freq.sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0]).take(30) #si los queremos sin frecuencias...(no es necesario para resolución)

['.',
 'i',
 ',',
 '!',
 'it',
 'game',
 'the',
 "'s",
 'good',
 'like',
 'app',
 'this',
 'great',
 'love',
 'get',
 'time',
 '...',
 "n't",
 '?',
 'would',
 'really',
 'even',
 'not',
 'ca',
 'update',
 'phone',
 'you',
 'work',
 "'m",
 'please']

Del total de tokens, obtenemos cuáles son stopwords filtrando. El map es evitable, simplemente lo hago para no mostrar la frecuencia de cada token. Puedo hacer un collect porque sé que las stopwords son un conjunto acotado de palabras.

In [ ]:
stopwords = set(stopwords.words('english'))

In [ ]:
tokens_by_freq.filter(lambda x: x[0] in stopwords).map(lambda x: x[0]).collect()

['i',
 'myself',
 'before',
 'this',
 'very',
 'in',
 'no',
 'her',
 'now',
 'but',
 'is',
 'than',
 'of',
 'there',
 'am',
 'do',
 'are',
 'just',
 'after',
 'was',
 'down',
 'when',
 'again',
 'them',
 'where',
 'below',
 'at',
 'more',
 'only',
 'we',
 'why',
 'out',
 'an',
 'have',
 'other',
 'don',
 'these',
 'once',
 'into',
 'until',
 'he',
 'as',
 'd',
 'own',
 'nor',
 'themselves',
 'his',
 'y',
 'both',
 's',
 'during',
 'above',
 'further',
 'against',
 'ourselves',
 'wasn',
 'couldn',
 'yourselves',
 'yours',
 've',
 'doesn',
 'theirs',
 'that',
 'because',
 'it',
 'not',
 'my',
 'she',
 'you',
 'up',
 'does',
 'its',
 'the',
 'can',
 'so',
 'to',
 'and',
 'be',
 'or',
 'same',
 'for',
 'by',
 'on',
 'has',
 'been',
 'me',
 'if',
 'a',
 'should',
 'which',
 'all',
 'what',
 'they',
 'few',
 'from',
 'with',
 'had',
 'most',
 'those',
 'then',
 'how',
 'did',
 'your',
 'too',
 'who',
 't',
 'will',
 'about',
 'such',
 'while',
 'doing',
 'any',
 'itself',
 'some',
 'over',
 

Por "culpa" de las abreviaciones nos quedan separados "n't" de "not", nos queda "'m" en vez de "am", "'s" en vez de "is" y algo similar sucede con "ca", que probablemente queda así porque el tokenizer separa "can't" literalmente en dos, en vez de considerar que realmente es "cannot".

Tanto am, is como can están entre las stopwords pero por aparecer abreviadas no se tienen en cuenta.

Not debería tener mayor frecuencia juntado los not y los n't.

Podría haber eliminado símbolos (como el punto o la coma, que quedaron entre los primeros) de las reviews para obtener resultados solo con palabras, pero no sé si era la idea del ejercicio ya que estos pueden ser considerados como tokens.